In [1]:
#Dependencies for Pyvoip
import pyVoIP
from pyVoIP.VoIP import CallState, VoIPPhone, InvalidStateError
import time
import wave

In [ ]:
#variables imported from kubernetes or network-specific
#pod name for file instance and grabbing credentials
pod = "pod1"
#sip extension
extension = "1001"
#sip extension password
extension_password = "password"
#myIP
IP = "x.x.x.x"
#Asterisk server ip
asteriskip = "x.x.x.x"
#Asterisk server trunk
trunkname = ""
#caller id
caller_id = ""
#context on asterisk
context = ""
#ftp target directory for callfile
ftp_directory

#mongo notes
uri = ""
db_name = ""
collection_name = ""

In [ ]:
#function whose purpose is to grab phone numbers from a MongoDB
#function for grabbing phone number, another ChatGPT creation
import pymongo
import random

def remove_random_phone_number(uri, db_name, collection_name):
    """
    Remove a random phone number from MongoDB and return the removed phone number.

    Parameters:
    - uri (str): The connection URI for MongoDB.
    - db_name (str): The name of the MongoDB database.
    - collection_name (str): The name of the collection in the database.

    Returns:
    - str or None: The removed phone number, or None if no phone number was removed.
    """
    # Connect to MongoDB
    myclient = pymongo.MongoClient(uri)

    # Access the specified database and collection
    mydb = myclient[db_name]
    mycol = mydb[collection_name]

    # Count the number of documents in the collection
    count = mycol.count_documents({})

    if count == 0:
        print("No phone numbers found in the database.")
        return None

    # Generate a random index
    random_index = random.randint(0, count - 1)

    # Find a random document from the collection
    random_document = mycol.find().skip(random_index).limit(1)[0]

    # Remove the random document from the collection
    result = mycol.delete_one({"_id": random_document["_id"]})

    if result.deleted_count == 1:
        print(f"Random phone number '{random_document['phone_number']}' removed successfully.")
        return random_document['phone_number']
    else:
        print("Failed to remove random phone number.")
        return None

In [ ]:
from ftplib import FTP
#call_making function, relies on the Asterisk callfile format
#grabs phone number from Asterisk db, then makes a call
def generate_call_file(filename, channel, context, extension, priority, caller_id, max_retry=0, retry_time=60):
    """
    Generate a call file for Asterisk PBX server.

    Parameters:
    - filename (str): The filename for the call file to be generated.
    - channel (str): The channel to use for the call (e.g., SIP/1001). Format SIP/trunkname/phone-number
    - context (str): The context in the dialplan to send the call to.
    - extension (str): The extension to dial.
    - priority (int): The priority in the dialplan to execute.
    - caller_id (str): The caller ID to display.
    - max_retry (int): Maximum number of retries in case of failure (default: 3).
    - retry_time (int): Time to wait between retries in seconds (default: 60).
    """
    #note this doesn't include the WaitTime parameter, which defaults to 45 seconds of waiting to see if call will answer. 
    with open(filename, 'w') as f:
        f.write(f"Channel: {channel}\n")
        f.write(f"Context: {context}\n")
        f.write(f"Extension: {extension}\n")
        f.write(f"Priority: {priority}\n")
        f.write(f"Callerid: \"{caller_id}\"\n")
        f.write(f"MaxRetries: {max_retry}\n")
        f.write(f"RetryTime: {retry_time}\n")

def upload_call_file_via_ftp(filename, ftp_host, ftp_user, ftp_pass, ftp_outgoing_dir):
    """
    Upload a call file to the Asterisk server's outgoing directory using FTP.

    Parameters:
    - filename (str): The filename of the call file to be uploaded.
    - ftp_host (str): The hostname or IP address of the FTP server.
    - ftp_user (str): The FTP username.
    - ftp_pass (str): The FTP password.
    - ftp_outgoing_dir (str): The path to the Asterisk server's outgoing directory.
    """
    with FTP(ftp_host) as ftp:
        ftp.login()
        ftp.cwd(ftp_outgoing_dir)
        with open(filename, 'rb') as file:
            ftp.storbinary(f'STOR {filename}', file)

def check_call_file_presence(filename, outgoing_dir):
    """
    Check if the call file is present in the outgoing directory after a set amount of time.

    Parameters:
    - filename (str): The filename of the call file to check.
    - outgoing_dir (str): The path to the Asterisk server's outgoing directory.
    - timeout (int): The maximum time to wait for the file to appear (default: 30 seconds).
    """
    ftp.login()
    file_list = ftp.nlst(outgoing_dir)
    for i in file_list:
        if i == pod+"call.call":
            return True
    return False

In [ ]:
#global variable for phone_number
current_phone = ''
#runs once when phone call starts, then either during final stage of successful phone call, or after hangup
#no_passthrough indicates loop. Use call_made = False and no_passthrough = False for predictive dial, call_made = True and no_passthrough = True for post-dial
def call_maker(no_passthrough,call_made):
    timer = 0
    warning_timer = 0
    if no_passthrough == False:
            current_phone = remove_random_phone_number(uri, db_name, collection_name)
            generate_call_file(pod+"call.call","SIP/"+trunkname+"/"+phone_number,context,extension,priority=1,caller_id)
            upload_call_file_via_ftp(pod+"call.call",asteriskip,ftp_directory)
    while call.state != CallState.ANSWERED and no_passthrough:
        if call_made = False:
            #function for grabbing phone number and writing it to remote file
            current_phone = remove_random_phone_number(uri, db_name, collection_name)
            generate_call_file(pod+"call.call","SIP/"+trunkname+"/"+phone_number,context,extension,priority=1,caller_id)
            upload_call_file_via_ftp(pod+"call.call",asteriskip,ftp_directory)
            call_made = True
        elif call_made = True and timer > 10:
            #checks to make sure file is present
            if check_file_presence(filename,outgoing_dir) == False:
                call_made = False
            else:
                warning_timer += 10
                timer = 0
        elif warning_timer > 100:
            call_made = False
        elif warning_timer > 200:
            raise TypeError("Asterisk Server Fault")
        else:
            time.sleep(0.1)
            timer += 0.1
    return
        #function for making call

In [ ]:
#Recording of call transcripts in my MongoDB
from datetime import datetime
import pymongo
uri = ""
myclient = pymongo.MongoClient(uri)
mydb = myclient["calltranscripts"]
#change cluster on column for future A/B testing
mycol = mydb["Cluster A"]

#separate do not call db
mydb1 = myclient["donotcall"]
mycol1 = mydb1["number_list"]

#this python function also thanks to ChatGPT
def push_transcript(transcript):
    if answering_machine < 2:
        is_machine = True
    else:
        is_machine = False
    transcript["is_machine"] = is_machine
    transcript["call_stage"] = call_stage
    transcript["datetime_start"] = datetime_start
    transcript["datetime_end"] = datetime.now()
    transcript["phone_number"] = phone_number
    x = mycol.insert_one(transcript)
    return

def do_not_call(phone_number):
    x = mycol1.insert_one(phone_number)
    return

In [2]:
#Dependencies for vad 
import wave
import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint

USE_ONNX = False # change this to True if you want to test onnx model
if USE_ONNX:
    !pip install -q onnxruntime
  
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=USE_ONNX)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

100%|██████████| 1.83M/1.83M [00:00<00:00, 2.69MB/s]
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to C:\Users\hmlay/.cache\torch\hub\master.zip


In [3]:
#Dependencies for faster-whisper
from faster_whisper import WhisperModel

model_size = "base.en"

full_model_size = "small.en"

#try compute_type float16 when implemented on arm

model = WhisperModel(model_size, device="cpu",compute_type="int8")

full_model = WhisperModel(full_model_size, device="cpu",compute_type="int8")

In [6]:
#global variable for current transcript
current_transcript = ''
full_transcript = ''

def whisper_audio_transcript(audiofile):
    segments, info = model.transcribe(audiofile, beam_size=5)
    segments = list(segments)  # The transcription will actually run here.
    transcript = ''
    for segment in segments:
        transcript += segment.text
    return(transcript)

#version for whisper_large

def full_whisper(audiofile):
    segments, info = full_model.transcribe(audiofile, beam_size=5)
    segments = list(segments)  # The transcription will actually run here.
    transcript = ''
    for segment in segments:
        transcript += segment.text
    return(transcript)
    

In [4]:
#Dependencies for RVC_Python and Silero TTS
from rvc_python.infer import infer_file, infer_files

#modify the below to change voice selection with Silero
language = 'en'
model_id = 'v3_en'
sample_rate = 48000
speaker = 'en_1'


import torch
device = torch.device('cpu')
model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=model_id)
model.to(device)

from pydub import AudioSegment

2024-02-05 16:42:01 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
Using cache found in C:\Users\hmlay/.cache\torch\hub\snakers4_silero-models_master


In [5]:
#VAD: takes in audio file and the current audio transcript, and determines whether our speaker should start talking. 

#global variable "talk_over_gap" is actively adjusted to ensure that the AI doesn't repeatedly talk over you
talk_over_gap = 0.2
def vad(audiofile,audio_text):
    wav = read_audio(audiofile, sampling_rate=8000)
    # get speech timestamps from full audio file
    speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=8000)
    last_speech = speech_timestamps[-1]
    with wave.open(audiofile,'rb') as wf:
        frames = wf.getnframes()
    vocal_gap = (frames - last_speech['end'])/8000
    #vocal_gap is measured in seconds. current target is the 200ms switching. incorporate variable in place of 0.2, etc in elif in order to get
    #dynamic response time. Currently reliant on Whisper's punctuation. 
    if vocal_gap > 0.1:
        tmp_audio_record = b''
        vadnum += 1
    if vocal_gap > talk_over_gap and audio_text[-1] in (".", "!", "?", " "):
        return True
    elif vocal_gap > talk_over_gap + 0.3:
        return True
    else:
        return False    


In [7]:
def my_tts(text2,destination_audiofile):
    #Silero TTS. Using SSML. 
    audio = model.save_wav(ssml_text=text2, speaker=speaker, sample_rate=sample_rate, audio_path=r"intermediate2.wav")
    sound = AudioSegment.from_wav(r"intermediate2.wav")

    # Convert to 8-bit, 8000Hz mono
    sound = sound.set_frame_rate(8000).set_channels(1).set_sample_width(1)

    # Export the converted audio
    sound.export(destination_audiofile, format="wav")


#full TTS function is available on tts_work ipynb, will likely deploy with Azure API for testing. Do try inference performance on Ampere Altra though. 

In [ ]:
#another ChatGPT-generated function. This one adds SSML tags, slowing speech down and adding breaks between sentences, and returns an array of sentences.
def separate_into_sentences_with_ssml(input_string):
    """
    Separate a string into an array of strings breaking at each sentence with SSML tagging.

    Parameters:
    - input_string (str): The input string.

    Returns:
    - list: An array of strings with SSML tagging.
    """
    # Split the input string into sentences
    sentences = input_string.split('. ')  # Assuming sentences end with period followed by space

    # Initialize the list to store SSML tagged sentences
    ssml_sentences = []

    # Iterate through each sentence and add SSML tagging
    for sentence in sentences:
        # Add SSML elements for strong pause and set prosody rate to slow
        ssml_sentence = f'<prosody rate="slow">{sentence.strip()}</prosody><break strength="strong"/>'
        ssml_sentences.append(ssml_sentence)

    return ssml_sentences

#and another ChatGPT python function that removes any pesky trailing sentences

def strip_sentences_without_punctuation(input_string):
    """
    Strip off sentences in a string that don't end with appropriate punctuation.

    Parameters:
    - input_string (str): The input string.

    Returns:
    - str: The modified string with only sentences ending with appropriate punctuation.
    """
    # Split the input string into sentences
    sentences = input_string.split('. ')  # Assuming sentences end with period followed by space

    # Initialize the list to store sentences with appropriate punctuation
    valid_sentences = []

    # Iterate through each sentence and check if it ends with appropriate punctuation
    for sentence in sentences:
        # Check if the sentence ends with ".", "!", or "?"
        if sentence.endswith(".") or sentence.endswith("!") or sentence.endswith("?"):
            valid_sentences.append(sentence)

    # Join the valid sentences to form the modified string
    modified_string = '. '.join(valid_sentences)

    return modified_string

In [8]:
#global variable for ongoing summary
#global variable for call stage

call_stage = 0

#dependencies, using openai API architecture

from openai import OpenAI

#API connection, in this case to deepinfra
openai = OpenAI(
    api_key="<YOUR DEEPINFRA TOKEN: deepctl auth token>",
    base_url="https://api.deepinfra.com/v1/openai",
)

#prompt that indicates what the system does, with list of one dictionary object indicating the start prompt
system_prompt = []

#prompt that indicates request is for filler sentence
filler_response = [{"role":"system","content":"Craft a short filler sentence to respond while you think."}]

#stage_responses

#introduction
stage_zero = [{"role":"system","content":""}]

#sales
stage_one = [{"role":"system","content":""}]

#getting info
stage_two = [{"role":"system","content":""]

#thank you
stage_three = [{"role":"system","content":""]

def script(transcript,response,call_stage):
    #hello!
    if call_stage == 0:
        chat_completion = openai.chat.completions.create(
        model="cognitivecomputations/dolphin-2.6-mixtral-8x7b",
        messages= system_prompt+transcript+stage_zero,
        stream=False, temperature = 1, presence_penalty = 1, frequency_penalty=-0.5,)
    #casual conversation
    elif: call_stage == 1:
        chat_completion = openai.chat.completions.create(
        model="cognitivecomputations/dolphin-2.6-mixtral-8x7b",
        messages= system_prompt+transcript+stage_one,
        stream=False, temperature = 1, presence_penalty = 1, frequency_penalty=-0.5,)
    #grabbing info
    elif: call_stage == 2:
        chat_completion = openai.chat.completions.create(
        model="cognitivecomputations/dolphin-2.6-mixtral-8x7b",
        messages= system_prompt+transcript+stage_two,
        stream=False, temperature = 1, presence_penalty = 1, frequency_penalty=-0.5,)
    #if info grabbed, polite close
    elif: call_stage == 3:
        chat_completion = openai.chat.completions.create(
        model="cognitivecomputations/dolphin-2.6-mixtral-8x7b",
        messages= system_prompt+transcript+stage_three,
        stream=False, temperature = 1, presence_penalty = 1, frequency_penalty=-0.5,)

    
    return chat_completion

def response(call):
    chat_completion = openai.chat.completions.create(
    model="cognitivecomputations/dolphin-2.6-mixtral-8x7b",
    messages= system_prompt+transcript+filler_response,
    stream=False, temperature = 1, presence_penalty = 1, frequency_penalty=-0.5,max_tokens=40)
    finished_response = strip_sentences_without_punctuation(chat_completion)
    return finished_response

#global variable for each call transcript
transcript = []

def update_transcript(transcript, caller_id, text):
    """
    Update the transcript with the provided caller ID and text.

    Parameters:
    - transcript (dict): The transcript dictionary.
    - caller_id (str): The caller ID.
    - text (str): The text message.

    Returns:
    - dict: The updated transcript.
    """

    # Create a dictionary element for the current update
    update_element = {"role": caller_id, "content": text}
    
    # Add the update element to the transcript
    transcript.append(update_element)

In [9]:
#function for writing audio file to call
def call_writer(audiofile):
    f = wave.open(audiofile, 'rb')
    frames = f.getnframes()
    data = f.readframes(frames)
    f.close()
    call.answer()
    call.write_audio(data)  # This writes the audio data to the transmit buffer, this must be bytes.
    #add function that sleeps for length of audio to prevent overwriting or building buffer
    time.sleep(frames/8000)

In [ ]:
#function for profanity and explicit detection
#another simple chatgpt creation

def profanity_checker(transcript):
    """
    Identify if a certain number of banned words occur in a transcript.

    Parameters:
    - transcript (list): The transcript list.
    - banned_words (list): The list of banned words.
    - threshold (int): The threshold count for banned words.

    Returns:
    - bool: True if the count of banned words exceeds the threshold, False otherwise.
    """
    # Initialize a counter for banned words
    banned_word_count = 0

    #banned words list
    #may extend
    banned_words = [] 
    
    #threshold

    threshold = 10

    # Iterate through each word in the transcript
    for sentence in transcript:
        for word in sentence.split():
            # Check if the word is in the list of banned words
            if word.lower() in banned_words:
                banned_word_count += 1

    # Check if the count of banned words exceeds the threshold
    if banned_word_count >= threshold:
        call.hangup()
        return
    else:
        return

    

In [10]:
#function for creation of white noise in appropriate frame/bytes format
#created by our good friend chatgpt, so could need tweaking
#consider running prior to create an index of, say, 10 minutes of frames from which we can grab randomly? 

def white_noise(master_file):
    with wave.open(master_file, 'rb') as wf:
        sample_width = wf.getsampwidth()  # Get the sample width in bytes
        frame_rate = wf.getframerate()  # Get the frame rate (e.g., 8000 Hz)
        num_frames = wf.getnframes()  # Get the total number of frames

        # Calculate the number of bytes per frame
        bytes_per_frame = sample_width

        # Set the desired length of the selection (160 frames)
        selection_length_frames = 160

        # Calculate the maximum frame index to start the selection from
        max_start_frame_index = num_frames - selection_length_frames

        # Generate a random start frame index within the valid range
        start_frame_index = random.randint(0, max_start_frame_index)

        # Move to the start frame index
        wf.setpos(start_frame_index)

        # Read the selected frames
        selected_frames = wf.readframes(selection_length_frames)

    return selected_frames


In [ ]:
#API interfaces with email and phone numbers

def send_text(phone_number):
    print(phone_number)

def send_email(email):
    print(email)



In [ ]:
#more chatgpt goodness, this identifies phone numbers using regex

import re

def spelled_out_to_digit(spelled_out_number):
    """
    Convert spelled out number to digit representation.

    Parameters:
    - spelled_out_number (str): Spelled out number (e.g., 'four', 'five', 'six').

    Returns:
    - str: Corresponding digit representation.
    """
    if spelled_out_number == 'zero':
        return '0'
    elif spelled_out_number == 'one':
        return '1'
    elif spelled_out_number == 'two':
        return '2'
    elif spelled_out_number == 'three':
        return '3'
    elif spelled_out_number == 'four':
        return '4'
    elif spelled_out_number == 'five':
        return '5'
    elif spelled_out_number == 'six':
        return '6'
    elif spelled_out_number == 'seven':
        return '7'
    elif spelled_out_number == 'eight':
        return '8'
    elif spelled_out_number == 'nine':
        return '9'
    else:
        return None

def contains_phone_number(sentence):
    """
    Identify if a ten-digit phone number is present in a sentence.

    Parameters:
    - sentence (str): The input sentence.

    Returns:
    - tuple: (bool, str) indicating if a ten-digit phone number is found and the phone number itself.
    """
    # Define regex pattern to match ten-digit phone numbers
    pattern = r'\b(?:\d{10})\b'

    # Find all matches of the pattern in the sentence
    matches = re.findall(pattern, sentence)

    # Check if any matches are found
    if matches:
        return True, matches[0]

    # Define regex pattern to match spelled out numbers followed by 'digit' or 'digits'
    spelled_out_pattern = r'(zero|one|two|three|four|five|six|seven|eight|nine) (?:digit|digits)'
    spelled_out_matches = re.findall(spelled_out_pattern, sentence)

    # Check if any spelled out matches are found and convert them to digits
    for spelled_out_match in spelled_out_matches:
        digit = spelled_out_to_digit(spelled_out_match[0])
        if digit:
            # Create a ten-digit phone number using the converted digit
            phone_number = digit * 10
            if phone_number in sentence:
                return phone_number

    return False

#even more chatgptness: 
def contains_email_address(sentence):
    """
    Identify if an email address is present in a sentence.

    Parameters:
    - sentence (str): The input sentence.

    Returns:
    - tuple: (bool, str) indicating if an email address is found and the email address itself.
    """
    # Define regex pattern to match email addresses
    pattern = r'\b([A-Za-z0-9._%+-]+)(?: at | @ )([A-Za-z0-9.-]+)\.[A-Z|a-z]{2,}\b'

    # Find all matches of the pattern in the sentence
    matches = re.findall(pattern, sentence)

    # Check if any matches are found
    if matches:
        # Concatenate the username and domain parts to form the email address
        email_address = f"{matches[0][0]}@{matches[0][1]}"
        return email_address

    return False

#stashes audio in the case that someone reads off an email address or whatever. 
def stash_audio(filename):



In [11]:
#defines threads of call while VAD is active
import threading
import time

#keeps track of ongoing response
my_response = ''
#creates empty bytes object
tmp_audio_record = b''
full_audio_record = b''

#keeps track of number of whisper chunks created by silero
vadnum = 0

#keeps track of which whisper chunks have been processed
whisper_increment = 0

def thread_1():
    while True:
        if vad('full_audio_record.wav',current_transcript) == True:
            return True
def thread_2():
    while True:
        full_audio_record += call.read_audio()
        tmp_audio_record += call.read_audio()
        with wave.open(str(vadnum)+'audio_record.wav', 'wb') as wf:
            wf.setnchannels(1)  # Set the number of channels (1 for mono, 2 for stereo)
            wf.setsampwidth(1)  # Set the sample width in bytes (2 bytes for 16-bit audio)
            wf.setframerate(8000)  # Set the frame rate (e.g., 44100 Hz)
            wf.writeframes(tmp_audio_record)  # Write the audio data to the file
        with wave.open('full_audio_record.wav', 'wb') as wf:
            wf.setnchannels(1)  # Set the number of channels (1 for mono, 2 for stereo)
            wf.setsampwidth(1)  # Set the sample width in bytes (2 bytes for 16-bit audio)
            wf.setframerate(8000)  # Set the frame rate (e.g., 44100 Hz)
            wf.writeframes(full_audio_record)  # Write the audio data to the file
def thread_3():
    while True:
        if vadnum <= whisper_increment:
            if tmp_current_transcript in current_transcript:
                current_transcript.replace(tmp_current_transcript,'')
            tmp_current_transcript = whisper_audio_transcript(str(vadnum)+'audio_record.wav')
            current_transcript += tmp_current_transcript
        elif vadnum > whisper_increment:
            tmp_current_transcript = whisper_audio_transcript(str(vadnum)+'audio_record.wav')
            current_transcript += tmp_current_transcript
            whisper_increment += 1
            

def thread_4():
    while True:
        my_response = response(current_transcript)
        my_ssml_response = """<speak> <prosody rate="slow">""" + my_response + """</prosody> </speak>"""
        my_tts(my_ssml_response,"response.wav")

def thread_5():
    while True:
        #uses the whitenoise.wav file as placeholder 
        call.write_audio(white_noise(master_file.wav))
        time.sleep(0.02)


        
        

In [13]:
#defines threads of call while VAD is inactive

#note other functions will take place before activation, inlcuding clearing local transcript and audiofile, along with summarization

#global variable to indicate llm_response completion
llm_response_finished = False
full_response = ''

#global variable indicating number of chunks full LLM response is in
index_count = 0
llm_responses_completed = 0 

def thread_a():
    while True:
        #adding 'a' so that continued audiofile compilation won't make sub-200ms judgements
        if vad('audio_record.wav',"a") == False:
            #set talked-over variable
            talk_over_gap += 0.3
            audio_record = ''
            return True
def thread_b():
    while llm_response_finished == False and llm_responses_completed < index_count:
        call_writer('response.wav')
        #setting global variable to indicate whether LLM has finished talking
        if llm_responses_completed < index_count:
            call_writer(str(llm_responses_completed)+'llmresponse.wav')
            llm_responses_completed += 1
        else:
            #uses the whitenoise.wav file as placeholder
            call.read_audio(white_noise('whitenoise.wav'))
            time.sleep(0.02)
    return
#reuse thread_2, needed for VAD
#edited to enable chunking, adding tmp file
def thread_d():
    while True:
        if vad('tmp_audio_record.wav',current_transcript) == True:
            return True

#runs full_whisper. transcript_update_success ensures in case of termination that partial transcript is still grabbed.
transcript_update_success = False
def thread_e():
        transcript = full_whisper("full_audio_record.wav")
        update_transcript(transcript,"user",current_transcript)
        transcript_update_success = True
        

def thread_c():
    #removing re-sts of thread due to latency issues
    #current_transcript = whisper_audio_transcript('audio_record.wav')
    full_response = script(current_transcript,my_response)
    response_chunks = separate_into_sentences_with_ssml(full_response)
    index_count = len(response_chunks)
    for i in response_chunks:
        my_tts(response_chunks,str(index(i))+'llmresponse.wav')
    llm_response_finished = True

    

In [14]:
#checks to ensure call is still active
def thread_z():
    #local timecount variable
    current_time = 0
    #whether we've attempted to raise the other end of the call
    if answering_machine == 1:
        call_raise = True
    else: 
        call_raise = False
    while True:
        if call.state != CallState.ANSWERED:
            return
        elif call_raise = False and current_time > 10:
            #if it's been longer than X length of time, we play inquiry
            my_response = response("Are you still there?")
            my_tts(my_response,'stillthere.wav')
            call_writer('stillthere.wav')
            current_time = 0
            call_raise = True
        elif call_raise = True and current_time > 10:
            #if have already tried to raise other end of call once, hang up
            call.hangup()
        else: 
            time.sleep(0.02)
            current_time += 0.02

    

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (1449828404.py, line 6)

In [16]:
#heart of the function

#note that thread_1 and thread_a may not be needed based on functionality of while loop, and are possibly redundant as a result

#note global variable answering_machine, which is set to 0, then to 1 after first circuit is complete. If it is an answering machine it will
#in all likelihood not complete a second circuit. Thus we deactivate raise_call during 1. 

answering_machine = 0


def answer(call):
    phone_number = current_phone
    #add function that records datetime.now() when callstate changes to ANSWERED at datetime_start
    datetime_start = datetime.now()
    while call.state == CallState.ANSWERED:
        #do not call function
        dtmf = call.get_dtmf()
        if dtmf == "8":
            do_not_call(phone_number)
            call.hangup()
            
        #waiting for other person to begin talking
        while vad(audiofile,transcript) == True:
            threading.Thread(target=thread_1).start()
            threading.Thread(target=thread_2).start()
            threading.Thread(target=thread_3).start()
            threading.Thread(target=thread_z).start()
        threading.Thread(target=thread_1).stop()
        threading.Thread(target=thread_2).stop()
        threading.Thread(target=thread_3).stop()
        threading.Thread(target=thread_z).stop()
    
        
        #while the other person is talking
        while vad(audiofile,transcript) == False:
            threading.Thread(target=thread_1).start()
            threading.Thread(target=thread_2).start()
            threading.Thread(target=thread_3).start()
            threading.Thread(target=thread_4).start()
            threading.Thread(target=thread_5).start()
        threading.Thread(target=thread_1).stop()
        threading.Thread(target=thread_2).stop()
        threading.Thread(target=thread_3).stop()
        threading.Thread(target=thread_4).stop()
        threading.Thread(target=thread_5).stop()
    
        #clears out variables
        full_audio_record = b''
        tmp_audio_record = b''
        
        #initiates new call before thank-you message
        if call_stage == 3:
            call_maker(False,False)
        
        #while our AI is talking
        while vad(audiofile,"a") == True and :
            threading.Thread(target=thread_d).start()
            threading.Thread(target=thread_a).start()
            threading.Thread(target=thread_b).start()
            threading.Thread(target=thread_c).start()
            threading.Thread(target=thread_e).start()
        threading.Thread(target=thread_e).stop()
        threading.Thread(target=thread_d).stop()
        threading.Thread(target=thread_a).stop()
        threading.Thread(target=thread_b).stop()
        threading.Thread(target=thread_c).stop()
        
        if transcript_update_success == False:
                update_transcript(transcript,"user",current_transcript)
        phonenum = contains_phone_number(current_transcript)
        if phonenum != False:
            send_text(phonenum)
        email = contains_email_address(current_transcript)
        if email != False:
            send_email(email)
        if phonenum or email != False:
            stash_audio("full_audio_record.wav")
        current_transcript = ''
        update_transcript(transcript,"assistant",(my_response + full_response))
        my_response = ''
        full_response = ''
        answering_machine += 1
        if answering_machine > 1:
            call_stage = 1
        if answering_machine > 3:
            call_stage = 2
        if call_stage == 3:
            call_maker(False,False)
        #reduces the gap by half its deviation from 200ms if a gap has been created
        if talk_over_gap > 0.2:
            talk_over_gap = talk_over_gap - (talk_over_gap-0.2)/2
    #checks to make sure wasn't hung up on do not call
    dtmf = call.get_dtmf()
    if dtmf == "8":
        do_not_call(current_phone)
    #pushes transcript to file
    push_transcript(transcript)
    call_maker(True,True)
    #resets global variables
    llm_response_finished = False
    full_response = ''
    #keeps track of ongoing response
    my_response = ''
    #creates empty bytes object
    tmp_audio_record = b''
    #global variable for each call transcript
    transcript = []
    #global variable for call stage
    call_stage = 0
    #global variable for current transcript
    current_transcript = ''
    full_transcript = ''
    answering_machine = 0
    datetime_start = ''
    index_count = 0
    llm_responses_completed = 0 
    tmp_audio_record = b''
    full_audio_record = b''
    transcript_update_success = False




SyntaxError: unterminated string literal (detected at line 15) (1630852504.py, line 15)

In [ ]:
#the actual pyvoip call
phone = VoIPPhone(asteriskip, 5060, extension, extension_passowrd,myIP=IP, callCallback=answer)
phone.start()
call_maker(False,False)
input('Press enter to disable the phone')
phone.stop()